In [ ]:
import pandas as pd
import numpy as np
import nltk
#nltk.download()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
comb_jobs = pd.read_csv('/content/datawithfieldcluster.csv')
user = pd.read_csv('/content/PFEfinal.csv')
Profil = pd.read_csv('/content/clusterTech.csv')

In [ ]:
comb_jobs = pd.read_csv('/content/datawithfieldcluster.csv')

In [ ]:
Profil = pd.read_csv('/content/clusterTech.csv')

In [ ]:
comb_jobs = comb_jobs.rename(columns={'Unnamed: 0': 'jobID'})
#user = user.rename(columns={'Unnamed: 0': 'ApplicantID'})

In [ ]:
#comb_jobs_df = comb_jobs[['jobID', 'Topic', 'Duration',  'Location', 'Field', 'Description','Technologies']]
comb_jobs_df=comb_jobs
comb_jobs_df.tail()

In [ ]:
comb_jobs_df['Technologies'] = comb_jobs_df['Technologies'].apply(str).astype(str)
comb_jobs_df['Technologies']= comb_jobs_df['Technologies'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)
comb_jobs_df['Text'] = comb_jobs_df['Topic'].map(str) + ' ' + comb_jobs_df['Technologies'] 
comb_jobs_all = comb_jobs_df[['jobID', 'Text', 'Topic','Technologies']]
nltk.download('stopwords')
stopword = stopwords.words('english')
stopword_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
comb_jobs_df.to_csv("PFEdatafinaloffers.csv")

In [ ]:
user['Text'] = user['headline'] + " " + user['allSkills']
user.head()

In [ ]:
Profil

In [ ]:
Profil['Technologies'] = Profil['Technologies'].apply(str).astype(str)
Profil['Technologies']= Profil['Technologies'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)
Profil['Technologies'] = Profil['Technologies'].apply(str).astype(str)
Profil['Topic'] = Profil['Topic'].apply(str).astype(str)
Profil['Topic']= Profil['Topic'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)

In [ ]:
Profil['Text'] = Profil['Technologies'] + " " + Profil['Topic']
Profil.head()

#### **Term Ferquency - Inverse Document Frequency**

In [ ]:
tfidf_vect = TfidfVectorizer()
# Fitting and transforming the vector
tfidf_comb = tfidf_vect.fit_transform((comb_jobs_all['Text'])) 
tfidf_comb

<397x1630 sparse matrix of type '<class 'numpy.float64'>'
	with 4497 stored elements in Compressed Sparse Row format>

In [ ]:
count_vect = CountVectorizer()
# Fitting and transforming the vectorizer
count_comb = count_vect.fit_transform((comb_jobs_all['Text'])) #fitting and transforming the vector
count_comb

<397x1630 sparse matrix of type '<class 'numpy.int64'>'
	with 4497 stored elements in Compressed Sparse Row format>

Creation de Base pour la Rcommandation 

#### **Building the Recommender Systems**

In [ ]:
def get_recommendation(top, comb_jobs_all, scores,user):
  recommendation = pd.DataFrame(columns = ['Applicant_ID', 'jobID',  'Topic', 'Score'])
  count = 0
  for i in top:
      recommendation.at[count, 'Applicant_ID'] = user
      recommendation.at[count, 'jobID'] = comb_jobs_all['jobID'][i]
      recommendation.at[count, 'Topic'] = comb_jobs_all['Topic'][i]
      recommendation.at[count, 'Score'] =  scores[count]
      count += 1
  return recommendation

In [ ]:
user

,ApplicantID,headline,fullName,allSkills,cluster,Text
0,0,student at the private university of engineeri...,Tasnim Naji,"VMware, Microservices, Packet Tracer, Logiciel...",Dev,student at the private university of engineeri...
1,1,student at the private engineering and technol...,Oussama Lachiheb,"Angular, Spring Boot, PHP, MySQL, Doctrine (PH...",Dev,student at the private engineering and technol...
2,2,private engineering and technology university ...,berzouga siwar,"Microsoft Word, SOLIDWORKS, Java, Spring Boot,...",Dev,private engineering and technology university ...
3,3,software engineering student at esprit,Rim Kourou,"Microsoft Power Platform, SharePoint, Communic...",Software,software engineering student at esprit Microso...
4,4,looking for a pfe internship | student at the ...,Majdi Msallem,"Laravel, Symfony, Feuilles de style en cascade...",Bi,looking for a pfe internship | student at the ...
...,...,...,...,...,...,...
457,486,full-stack web developer (angular/reactjs/spri...,Adem DAAMI,"Angular, Bootstrap, JIRA, Scrum, materialui, G...",Software,full-stack web developer (angular/reactjs/spri...
458,487,student at private high school of engineering ...,Eya Ben Amor,"Angular, Enjoy Challenges, Communication, Prob...",Software,student at private high school of engineering ...
459,488,engineering student at esprit | looking for a ...,Yasmine Grati,"Gestion de projet, DBeaver, Travail d’équipe, ...",Bi,engineering student at esprit | looking for a ...
460,490,it engineering student at esprit university sp...,Eyatallah THEBTI,"Développement web, Développement full-stack, D...",Bi,it engineering student at esprit university sp...


In [ ]:
test_jobSeekers=user.tail(30)

In [ ]:
test_jobSeekers

In [ ]:
N = len(user)

In [ ]:
user = user.drop(user.index[-30:])

In [ ]:
user

In [ ]:
all_dataframes=[]
for i in range(len(user)):
    # Select the row using iloc
    row = user.iloc[i]

    # Create a DataFrame for the row
    row_df = pd.DataFrame(row).T  # Transpose the row to convert it to a column
    row_df.columns = [f'{col}' for col in row_df.columns]  # Rename the columns to avoid duplicates
    user_count_segheweb = count_vect.transform([row_df['Text'].tolist()[0]])
    cos_sim_count_seghe = map(lambda x: cosine_similarity(user_count_segheweb, x), count_comb)
    rem1 = list(cos_sim_count_seghe)
    top10_seghe_count = sorted(range(len(rem1)), key = lambda i: rem1[i], reverse = True)[:10]
    list_scores_seghe_count = [rem1[i][0][0] for i in top10_seghe_count]
    df = get_recommendation(top10_seghe_count, comb_jobs_all, list_scores_seghe_count,[row_df['ApplicantID'].tolist()[0]])
    all_dataframes.append(df)
concatenated_df = pd.concat(all_dataframes, axis=0)

In [ ]:
DFPFERecommandation=concatenated_df

In [ ]:
DFPFERecommandation

,Applicant_ID,jobID,Topic,Score
0,[0],362,microsoft azure governanceoverviewmicrosoft se...,0.31414
1,[0],206,implementation authentication mode topic sso ...,0.307794
2,[0],207,developing microsoft power app topic apps cus...,0.287019
3,[0],360,intelligent chatbots microsoft azure overviewm...,0.276531
4,[0],364,implementation process management apptopic imp...,0.269953
...,...,...,...,...
5,[459],179,design data quality framework,0.333914
6,[459],183,establishment recruitment management system,0.331295
7,[459],184,lms learning management system,0.331295
8,[459],23,association management,0.312348


In [ ]:
DFPFERecommandation['Applicant_ID'] = DFPFERecommandation['Applicant_ID'].apply(str).astype(str)
DFPFERecommandation['Applicant_ID']= DFPFERecommandation['Applicant_ID'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)

In [ ]:
DFPFERecommandation

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(user['Text'])
dfuser = user.reset_index()
userid = user['ApplicantID']
indices = pd.Series(user.index, index=user['ApplicantID'])
indices.head(2)
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# **Profil recommendation**

In [ ]:
def get_recommendationProfil(top, comb_jobs_all, scores,Profil):
  recommendation = pd.DataFrame(columns = ['Profil', 'jobID',  'Topic', 'Score'])
  count = 0
  for i in top:
      recommendation.at[count, 'Profil'] = Profil
      recommendation.at[count, 'jobID'] = comb_jobs_all['jobID'][i]
      recommendation.at[count, 'Topic'] = comb_jobs_all['Topic'][i]
      recommendation.at[count, 'Score'] =  scores[count]
      count += 1
  return recommendation

In [ ]:
Profil

,Unnamed: 0,Field,Technologies,Location,Company,Duration,Topic,Text
0,0,data engineering,"french, french english, bi, typescript, lan & ...",['tunis' 'sousse' 'ariana'],['Sofrecom' 'ODDO' 'DRAXLMAIER' 'AxeFinance' '...,6,implementation data mesh architecture develop...,"french, french english, bi, typescript, lan & ..."
1,1,software development,"french, vuejs, french english, orm nhibernate,...",['sousse' 'tunis'],['InSoDev' 'Keyrus' 'Sofrecom' 'ODDO' 'AIVENTU...,6,delivery optimization service\n integration da...,"french, vuejs, french english, orm nhibernate,..."
2,2,data science,"french, french english, bi, opencv, k8s, bpmn,...",['sousse' 'tunis'],['DueToData' 'Addin' 'Keyrus' 'Sofrecom' 'ODDO...,6,model analysis model management application mo...,"french, french english, bi, opencv, k8s, bpmn,..."
3,3,business intelligence,"french english, bi, etl, c++, flutter, android...",['tunis' 'nan' 'ariana'],['Sofrecom' 'ODDO' 'AIVENTU' 'AxeFinance' 'COF...,6,development application mobile behalf employe...,"french english, bi, etl, c++, flutter, android..."
4,4,cloud computing,"packer, jenkins, gitlab, github, rest apis, go...",['sousse' 'sfax' 'tunis'],['InSoDev' 'Fogits' 'Addin' 'Relead' 'Sofrecom...,from 4 to 6 (face-to-face),automation optimization deployment migration a...,"packer, jenkins, gitlab, github, rest apis, go..."
5,5,web development,"french english, bi, gitlab, operating system, ...",['sousse' 'tunis' 'monastir' 'ariana'],['InSoDev' 'Keyrus' 'Sofrecom' 'WeMiam' 'ATS' ...,4,microservice parsing edi path calculation serv...,"french english, bi, gitlab, operating system, ..."
6,6,embedded development,"compiler, eclipse, teststand, c++, labwindows,...",['ariana' 'sfax' 'tunis'],['ACTIA' 'S2T'],4 to 6,development infotainment system agl automotive...,"compiler, eclipse, teststand, c++, labwindows,..."
7,7,research & development,"bi, workflow, typescript, rest apis, flutter, ...",['sfax' 'tunis'],['Fogits' 'EXPENSYA' 'AxeFinance'],6,ai chatbot voice text rfid access control\n...,"bi, workflow, typescript, rest apis, flutter, ..."
8,8,business management,"english, french, bi, database design, amplitud...",['tunis' 'sousse' 'siliana'],['Sofrecom' 'DRAXLMAIER' 'EXPENSYA' 'AxeFinance'],4,design creation assistant generic creation fo...,"english, french, bi, database design, amplitud..."
9,9,ai,"blockchains, c++, linux, solidity, deep learni...",['tunis' 'ariana'],['Sofrecom' 'AxeFinance' 'Capgemini'],4-6,development generation solution audio speech ...,"blockchains, c++, linux, solidity, deep learni..."


In [ ]:
all_dataframesProfil=[]
for i in range(len(Profil)):
    # Select the row using iloc
    row = Profil.iloc[i]

    # Create a DataFrame for the row
    row_df = pd.DataFrame(row).T  # Transpose the row to convert it to a column
    row_df.columns = [f'{col}' for col in row_df.columns]  # Rename the columns to avoid duplicates
    user_count_segheweb = count_vect.transform([row_df['Field'].tolist()[0]])
    cos_sim_count_seghe = map(lambda x: cosine_similarity(user_count_segheweb, x), count_comb)
    rem1 = list(cos_sim_count_seghe)
    top10_seghe_count = sorted(range(len(rem1)), key = lambda i: rem1[i], reverse = True)[:10]
    list_scores_seghe_count = [rem1[i][0][0] for i in top10_seghe_count]
    df = get_recommendationProfil(top10_seghe_count, comb_jobs_all, list_scores_seghe_count,[row_df['Field'].tolist()[0]])
    all_dataframesProfil.append(df)
concatenated_dfPROFIL = pd.concat(all_dataframesProfil, axis=0)

In [ ]:
concatenated_dfPROFIL['Profil'] = concatenated_dfPROFIL['Profil'].apply(str).astype(str)
concatenated_dfPROFIL['Profil']= concatenated_dfPROFIL['Profil'].replace(to_replace=r'[\[\]\'"]', value='', regex=True)

In [ ]:
concatenated_dfPROFIL

,Profil,jobID,Topic,Score
0,data engineering,92,implementation data mesh architecture develop...,0.424264
1,data engineering,347,end study internships controlling,0.392232
2,data engineering,348,end study internships purchasing,0.392232
3,data engineering,345,end study internships logistics supply chain,0.365148
4,data engineering,63,setting hybrid sid azure data services,0.316228
...,...,...,...,...
5,"electronic , mechanical engineering",365,table contentwho industries valuespfe topicswh...,0.131622
6,"electronic , mechanical engineering",0,microservice parsing edi,0.0
7,"electronic , mechanical engineering",1,delivery optimization service,0.0
8,"electronic , mechanical engineering",2,path calculation service,0.0


In [ ]:
Profil.reset_index(drop=False, inplace=True)
Profil=Profil.drop('Unnamed: 0',axis=1)

FIND PROFIL SIMILRITY 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def find_user_profile(user_text, profile_data):
    # Create CountVectorizer object to convert text to vectors
    vectorizer = CountVectorizer()

    # Fit the vectorizer to the user text
    vectorizer.fit_transform([user_text])

    # Get the vector for the user text
    user_vector = vectorizer.transform([user_text]).toarray()[0]

    # Compute the cosine similarity between the user vector and each profile vector
    similarity_scores = []
    for    index , row  in profile_data.iterrows():
        profile_text =  row['Field']
        profile_vector = vectorizer.transform([profile_text]).toarray()[0]
        similarity_scores.append(cosine_similarity([user_vector, profile_vector])[0, 1])

    # Find the profile with the highest similarity score
    best_match_index = similarity_scores.index(max(similarity_scores))
    field_string = profile_data.loc[profile_data['index']==best_match_index, "Field"].to_string(index=False)
    field_string=field_string.replace("'","")
    

    return  field_string


In [ ]:
test_jobSeekers

In [ ]:
Text=test_jobSeekers.loc[test_jobSeekers['ApplicantID'] == 478, 'headline'].values

In [ ]:
Job_Seeker_Test=test_jobSeekers.loc[test_jobSeekers['ApplicantID'] == 478]

In [ ]:
Job_Seeker_Test

,ApplicantID,headline,fullName,allSkills,cluster,Text
450,478,business intelligence student,Aymen Saidi,"Python (Programming Language), Talend Open Stu...",Bi,business intelligence student Python (Programm...


In [ ]:
ProfilRedict=find_user_profile(str(Text), Profil)

In [ ]:
ProfilRedict

'business intelligence'

collaborative recommendations

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
applicant_vector = tf.fit_transform((user['Text'])) 
applicant_vector

<432x8033 sparse matrix of type '<class 'numpy.float64'>'
	with 30411 stored elements in Compressed Sparse Row format>

In [ ]:
jobS_record_vector = tf.transform(Job_Seeker_Test['Text'])
candidate_cos_similarity = map(lambda x: cosine_similarity(jobS_record_vector, x),applicant_vector)

In [ ]:
candidate_score = list(candidate_cos_similarity)

In [ ]:
candidate_score

In [ ]:
Job_Seeker_Test['ApplicantID']

450    478
Name: ApplicantID, dtype: int64

In [ ]:
user[user['ApplicantID']==532]["headline"]

Series([], Name: headline, dtype: object)

In [ ]:
user.head()

In [ ]:
def get_candidates(top, df, scores):
  print(top)
  candidates = pd.DataFrame(columns = ['NewJobSeeks', 'ApplicantID', 'keywords', 'score'])
  count = 0
  for i in top:
    print(i)
    candidates.at[count, 'NewJobSeeks'] = 1
    candidates.at[count, 'ApplicantID'] = df[df['ApplicantID'==i]]
    candidates.at[count, 'score'] =  scores[count]
    count += 1
  return candidates

In [ ]:
top_candidates = sorted(range(len(candidate_score)), key=lambda i: candidate_score[i], reverse=True)[:10]
list_scores = [candidate_score[i][0][0] for i in top_candidates]
user_userwise = user.index.isin(top_candidates)
df1 = pd.DataFrame(data = user[user_userwise], columns=['ApplicantID','headline','allSkills'])
ApplicantList = pd.DataFrame(data = user[user_userwise], columns=['ApplicantID'])


In [ ]:
df1

,ApplicantID,headline,allSkills
127,135,student at private engineering and technology ...,"Communication, Anglais, Finance, Business Inte..."
178,189,business intelligence student at esprit,"PostgreSQL, Talend, Microsoft Power BI"
248,262,applied mathematics student at esprit school o...,"Machine Learning, Artificial Intelligence (AI)..."
265,280,business analytics student | business intellig...,"Business Analytics, Data Visualization, Busine..."
267,282,business intelligence student,"Microsoft Power BI, MySQL, PHP, Cascading Styl..."
276,291,business intelligence student,"Python (Programming Language), Talend Open Stu..."
277,292,business intelligence engineering student,"Google Analytics, R (Programming Language), My..."
283,300,m1 business analytics student,"RStudio, Talend Open Studio, Alteryx BI + Visu..."
288,306,business intelligence student,"English, Talend Open Studio, Microsoft Power B..."
427,455,applied mathematics student at esprit school o...,"Machine Learning, Artificial Intelligence (AI)..."


In [ ]:
ApplicantList=ApplicantList['ApplicantID'].to_list()

In [ ]:
ApplicantList

[135, 189, 262, 280, 282, 291, 292, 300, 306, 455]

 **Content Recommendation**

In [ ]:
user_tfidf_gbade = tfidf_vect.transform(Job_Seeker_Test['headline'])
cos_sim_tfidf_gbade = map(lambda x: cosine_similarity(user_tfidf_gbade, x), tfidf_comb)

In [ ]:
rec1 = list(cos_sim_tfidf_gbade)

In [ ]:
top10_seghe_tfidf = sorted(range(len(rec1)), key = lambda i: rec1[i], reverse = True)[:10]
list_scores_seghe_tfidf = [rec1[i][0][0] for i in top10_seghe_tfidf]

In [ ]:
job_userwise = comb_jobs_df.index.isin(top10_seghe_tfidf)
dfcontent = pd.DataFrame(data = comb_jobs_df[job_userwise], columns=['jobID'])
ContenttList=dfcontent['jobID'].to_list()

In [ ]:
comb_jobs

In [ ]:
Job_list = comb_jobs['jobID'].isin(ContenttList) 
df_temp = pd.DataFrame(data = comb_jobs[Job_list], columns=['jobID','Technologies','Field','Topic'])
df_temp = df_temp.drop_duplicates()

In [ ]:
df_temp

,jobID,Technologies,Field,Topic
55,55,['ibm'],data science,system design development business intelligence
56,56,['sap'],data science,system design development business intelligence
57,57,"['talend', 'tableau']",data science,system design development business intelligence
58,58,"['talend', 'qlik']",data science,system design development business intelligence
59,59,['qlik'],data science,system design development business intelligence
351,351,"['english', 'ms office']",business intelligence,end study internships business intelligence
352,352,"['english', 'ms office']",business intelligence,end study internships business intelligence
353,353,"['english', 'ms office', 'data visualization']",software testing,end study internships business intelligence
355,355,"['english', 'ms office', 'machine learning']",data science,end study internships business intelligence
356,356,"['english', 'ms office', 'machine learning']",data science,end study internships business intelligence


Hybrid Recommendation


In [ ]:
def jobs_UserRecommendation(usrid_list):
    jobs_userwise = DFPFERecommandation['Applicant_ID'].astype(int).isin(usrid_list)
    df1 = pd.DataFrame(data = DFPFERecommandation[jobs_userwise], columns=['jobID'])
    joblist = df1['jobID'].tolist()
    return joblist

In [ ]:
def jobs_Profilecommendation(Profil):
  jobs_userwise=concatenated_dfPROFIL[concatenated_dfPROFIL['Profil']==Profil]['jobID'].tolist()
  return jobs_userwise

In [ ]:
ListUser=jobs_UserRecommendation(ApplicantList)

In [ ]:
ListProfil=jobs_Profilecommendation(ProfilRedict)

In [ ]:
result_list = list(set(ListProfil + ListUser + ContenttList))

In [ ]:
Job_list = comb_jobs['jobID'].isin(result_list) 
df_temp = pd.DataFrame(data = comb_jobs[Job_list], columns=['jobID','Topic','Field','Technologies'])
df_temp = df_temp.drop_duplicates()

In [ ]:
df_temp

,jobID,Topic,Field,Technologies
5,5,integration data management tool microservice ...,software development,"['linux', 'web', 'microservice', 'data science..."
23,23,association management,business performance,"['python', 'sql']"
24,24,hr dashboard,business performance,"['python', 'sql', 'bi']"
40,40,setting architecture storage etl,data science,"['bi', 'microsoft', 'sql', 'server']"
48,48,data maintenance predictive,software engineering,"['machine learning', 'python', 'angular']"
55,55,system design development business intelligence,data science,['ibm']
56,56,system design development business intelligence,data science,['sap']
57,57,system design development business intelligence,data science,"['talend', 'tableau']"
58,58,system design development business intelligence,data science,"['talend', 'qlik']"
59,59,system design development business intelligence,data science,['qlik']


**insuffisance des competences**



In [ ]:
SKILLS1 = Job_Seeker_Test['allSkills'].apply(lambda x: x.lower()).values
SKILLS1 = ', '.join(str(skill) for skill in SKILLS1)

In [ ]:
ColaborativeSkill = ','.join(df1.groupby(level=0)['allSkills'].apply(lambda x: ','.join([f.lower() for f in x])).values)
ColaborativeSkill = ','.join(set(ColaborativeSkill.split(',')))

In [ ]:
SKILLS1

'python (programming language), talend open studio, microsoft power bi, nosql, sql, english'

In [ ]:
ColaborativeSkill

In [ ]:
# split string1 into a list of elements
elements = ColaborativeSkill.split(',')
# extract elements from the list that are not in string2
result = [elem for elem in elements if elem not in SKILLS1]

In [ ]:
result_string = ', '.join(result)


In [ ]:
result_string

' etl tools,  eclipse,  tableau,  cloudera, communication,  javascript,  data mining,  sql server management studio (ssms),  back office operations,  java,  data analysis,  kpi dashboards,  deep learning, postgresql,  reporting, machine learning,  load (etl),  data warehousing,  business intelligence,  ibm spss,  compétences analytiques,  c (programming language),  html5,  pl/sql,  transform,  management,  webmaster,  intellij idea,  scrum,  emotional intelligence,  anglais,  unified modeling language (uml),  e-commerce,  webmastering,  cascading style sheets (css),  odoo,  r (programming language),  python (programming language),  mysql,  database management system (dbms),  python (langage de programmation),  databases,  gestion de projet,  data warehouse architecture,  microsoft office,  extract, business analytics,  parler en public,  php,  rstudio,  finance d’entreprise,  informatique décisionnelle,  postman api,  android studio,  finance,  openerp,  alteryx bi + visualization solu

In [ ]:
!pip install skillNer
!python -m spacy download en_core_web_lg

In [ ]:
# imports
import spacy
import en_core_web_lg
from spacy.matcher import PhraseMatcher

# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor
from skillNer.general_params import SKILL_DB
# init params of skill extractor
nlp = en_core_web_lg.load()
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...
